<a href="https://colab.research.google.com/github/bhatt01tapasvi1/PIV-Vicsek-Model-Nature-SR-paper-/blob/main/Vicsek_Model_Implementation1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Defining the variables:
L = 100 #Side length of the square
N = 500 #Number of particles
v0 = 1 #Speed of particles (1 unit/1 unit (say))
T = 500 #Total time for which simulation will run

#Defining the positions and orientations:
import numpy as np
x = np.random.uniform(0,L,N)
y = np.random.uniform(0,L,N)
theta = np.random.uniform(0,2*np.pi,N)

print(x[:5])
print(y[:5])
print(theta[:5])

[82.00529814 68.41587898 30.29915947 62.16770635 91.24792031]
[61.1814474  66.80929955 19.72547993 74.7146906  51.72592279]
[5.17042261 2.66759725 5.92919416 2.44366153 3.3788447 ]


In [ ]:
#I am doing two things here:
#1. At each time step, updating the position of particles
#2. Saving the positions in the list x_history and y_history

x_history = []
y_history = []

for t in range(T):
  x_history.append(x.copy())  #Copy the array to avoid overwriting
  y_history.append(y.copy())
  for i in range(N):
      x[i] = x[i] + v0*np.cos(theta[i])
      y[i] = y[i] + v0*np.sin(theta[i])
      if x[i] > L:
          x[i] = x[i] - L
      if y[i] > L:
          y[i] = y[i] - L
      if x[i] < 0:
          x[i] = x[i] + L
      if y[i] < 0:
          y[i] = y[i] + L

In [ ]:
#FIRST VIDEO WITHOUT PERTURBATION AND AVERAGING OF THETA

import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Create a figure and axis
fig, ax = plt.subplots()
ax.set_xlim(0, L)
ax.set_ylim(0, L)

# Initialize a scatter plot for particles
scat = ax.scatter(x_history[0], y_history[0])

# Function to update the scatter plot at each frame
def update(frame):
    scat.set_offsets(np.c_[x_history[frame], y_history[frame]])  # Update positions
    return scat,

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=T, interval=50, blit=True)

# Save the animation as an MP4 file
ani.save('particle_motion.mp4', writer='ffmpeg', fps=30)

from google.colab import files
files.download('particle_motion.mp4')

In [ ]:
r = 2 #Radius of the circle in which we will take the average
Sphere_Particles = [[] for _ in range(N)] #List of list which contains all those particles (j) which come in the sphere of particle (i)
for i in range(N):
  for j in range(N):
    if np.sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2) < r:
      Sphere_Particles[i].append(j)

print(Sphere_Particles[:15])

[[0, 78, 334], [1, 116, 175, 261], [2], [3], [4, 458], [5, 438], [6], [7, 328], [8, 355], [9, 345], [10, 13, 58, 327], [11], [12], [10, 13], [14, 482]]


Till now, we have created a simple animation of particles moving inside the frame with fixed velocities, and we can visualise that. And then we move to the real implementation of the Vicsek Model where we take a radius r and find all those inside the sphere.

Actually, as of now, I should have also updated the theta at each time interval also but let's go step by step, even if we repeat the work again, its fine. Because let's work on understanding how this thing works.

**NOW LET'S DO THINGS AGAIN, WITH UPDATING THETA ALSO** 😀

In [ ]:
# Initialize a list to store the average angles <θi(t)>r for each particle
average_theta_r = np.zeros(N)

# Loop over each particle i
for i in range(N):
    neighbors = Sphere_Particles[i]  # List of neighbors for particle i

    if len(neighbors) > 0:  # Only compute if there are neighbors
        sum_sin = np.sum(np.fromiter((np.sin(theta[j]) for j in neighbors), dtype=float))  # Sum of sine of neighbors' angles
        sum_cos = np.sum(np.fromiter((np.cos(theta[j]) for j in neighbors), dtype=float))  # Sum of cosine of neighbors' angles

        # Compute the arctangent of the sum of sin and cos to get the average angle
        average_theta_r[i] = np.arctan2(sum_sin, sum_cos)

    else:
        # If no neighbors, set average angle to the current particle's angle
        average_theta_r[i] = theta[i]

# Now average_theta_r contains the average angle for each particle
print(average_theta_r[:5])  # Print first 5 average angles to verify

[ 0.69224278  1.76515404 -2.3310368   2.96076166 -2.76472969]


In [ ]:
theta_history = []
eta = 5 #Parameter of perturbation

for t in range(T-1):
    average_theta_r[t+1] = average_theta_r[t] + np.random.uniform(-eta/2, eta/2)
    theta_history.append(average_theta_r.copy())

print(average_theta_r[:5])  # Print first 5 average angles to verify

[0.69224278 1.09409333 3.34017649 4.82491489 5.04771376]


In [ ]:
#SECOND VIDEO WITH PERTURBATION AND AVERAGING OF THETA

perturbed_x_history = []
perturbed_y_history = []

for t in range(T):
  perturbed_x_history.append(x.copy())  #Copy the array to avoid overwriting
  perturbed_y_history.append(y.copy())
  for i in range(N):
      x[i] = x[i] + v0*np.cos(average_theta_r[i])
      y[i] = y[i] + v0*np.sin(average_theta_r[i])
      if x[i] > L:
          x[i] = x[i] - L
      if y[i] > L:
          y[i] = y[i] - L
      if x[i] < 0:
          x[i] = x[i] + L
      if y[i] < 0:
          y[i] = y[i] + L

import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Create a figure and axis
fig, ax = plt.subplots()
ax.set_xlim(0, L)
ax.set_ylim(0, L)

# Initialize a scatter plot for particles
scat = ax.scatter(perturbed_x_history[0], perturbed_y_history[0])

# Function to update the scatter plot at each frame
def update(frame):
    scat.set_offsets(np.c_[perturbed_x_history[frame], perturbed_y_history[frame]])  # Update positions
    return scat,

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=T, interval=50, blit=True)

# Save the animation as an MP4 file
ani.save('perturbed_particle_motion.mp4', writer='ffmpeg', fps=30)

from google.colab import files
files.download('perturbed_particle_motion.mp4')

NameError: name 'files' is not defined

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>